In [70]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime
import pandas as pd
import enchant
import numpy as np
from nltk.stem.snowball import SnowballStemmer
stemmer = LancasterStemmer()
df_train = pd.read_csv("/home/yassaman/git/CMPE297/FinalProject/test_mini.csv").fillna("empty")
df_train.question1 = df_train.question1.astype(str)
df_train.question2 = df_train.question2.astype(str)

In [71]:
import string
import re
from string import digits
words = []
document1 = []
document2 = []
for index, row in df_train.iterrows():
    row['question1'].translate(None, string.punctuation)
    row['question2'].translate(None, string.punctuation)
    row['question1'] = re.sub(r"[^A-Za-z0-9]", " ", row['question1'])
    row['question2'] = re.sub(r"[^A-Za-z0-9]", " ", row['question2'])
    row['question1'] = re.sub('[!\[\]@#\\\.?,$/]', '', row['question1'])
    row['question2'] = re.sub('[!\[\]@#\\\.?,$/]', '', row['question2'])
#     remove_digits = str.maketrans('', '', digits)
    row['question1'] = row['question1'].translate(None, digits)
    row['question2'] = row['question2'].translate(None, digits)
    word_list = nltk.word_tokenize(row['question1'])
    word_list2 = nltk.word_tokenize(row['question2'])
    # add to our words list
    for w in word_list:
        w = SnowballStemmer("english").stem(w)
        words.append(w)
        # add to documents in our corpus
        document1.append(w)
    for w2 in word_list2:
        w2 = SnowballStemmer('english').stem(w2)
        words.append(w2)
        # add to documents in our corpus
        document2.append(w2)
words = list(set(words))
import enchant
d = enchant.Dict("en_US")
for w in words:
    if(d.check(w)==False):
        if(d.suggest(w) is not None):
            try:
                w = d.suggest(w)[0]
            except IndexError:
                w=w
                print 'Error ' + w 

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
from nltk.stem.snowball import SnowballStemmer
np.seterr(divide='ignore', invalid='ignore')
stemmer = LancasterStemmer()
def words_in_string(word_list, a_string):
    return set(word_list).intersection(a_string.split())
def cosine(a0,a1):
    dotp = np.dot(a0, a1)
    mag0 = np.linalg.norm(a0)
    mag1 = np.linalg.norm(a1)
    cosineSim = dotp / (mag0* mag1)
    return cosineSim
cosinesimilarityarray=[]
# print document1
# print '*****'
# print document2
for a,b in zip(document1,document2):
#     print a + ' , ' + b
    corpus = []
    newCorpus = []
    for i,j in zip(a,b):
#         print '----'
#         print a
        i = SnowballStemmer("english").stem(i)
        j = SnowballStemmer("english").stem(j)
#     print a[0]
    corpus.append(a[0])
    corpus.append(b[0])
    corpus[0] = ' '.join(corpus[0]).lower()
    corpus[1] = ' '.join(corpus[1]).lower()
    punc_trans_table = {ord(c): None for c in string.punctuation}
    corpus[0].translate(punc_trans_table)
    corpus[1].translate(punc_trans_table)
    corpus[0]= re.sub('[!\[\]@#\\\.:?)(}{,$/]', '', corpus[0])
    corpus[1]= re.sub('[!\[\]@#\\\.:?)(}{,$/]', '', corpus[1])
#     remove_digits = string.maketrans('', '', digits)
    digit_trans_table = {ord(c): None for c in string.digits}
    corpus[0] = corpus[0].translate(digit_trans_table)
    corpus[1] = corpus[1].translate(digit_trans_table)
#     string0 = words_in_string(words,corpus[0])
#     string1 = words_in_string(words,corpus[1])
    newCorpus.append(corpus[0])
    newCorpus.append(corpus[1])
    vectorizer = CountVectorizer(min_df=1) 
    analyze = vectorizer.build_analyzer()
    print newCorpus
    X = vectorizer.fit_transform(newCorpus)
    xArray = X.toarray()
    cosinesimilarity = cosine(xArray[0],xArray[1])
    cosinesimilarityarray.append(cosinesimilarity)

[u'what', 'is', u'the', u'step', 'by', u'step', u'guid', 'to', u'invest', 'in', u'share', u'market', 'in', u'india', u'what', 'is', u'the', u'stori', 'of', u'kohinoor', u'koh', 'i', u'noor', u'diamond', u'how', u'can', 'i', u'increas', u'the', u'speed', 'of', 'my', u'internet', u'connect', u'while', u'use', 'a', u'vpn', u'whi', 'am', 'i', u'mental', u'veri', u'lone', u'how', u'can', 'i', u'solv', 'it', u'which', u'one', u'dissolv', 'in', u'water', u'quik', u'sugar', u'salt', u'methan', u'and', u'carbon', 'di', u'oxid', u'astrolog', 'i', 'am', 'a', u'capricorn', u'sun', u'cap', u'moon', u'and', u'cap', u'rise', u'what', u'doe', u'that', u'say', u'about', 'me', u'should', 'i', u'buy', u'tiago', u'how', u'can', 'i', 'be', 'a', u'good', u'geologist', u'when', 'do', u'you', u'use', u'instead', 'of', u'motorola', u'compani', u'can', 'i', u'hack', 'my', u'charter', u'motorolla', u'dcx', u'method', 'to', u'find', u'separ', 'of', u'slit', u'use', u'fresnel', u'biprism', u'how', 'do', 'i', u'rea

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
len(cosinesimilarityarray)

In [ ]:
import enchant
d = enchant.Dict("en_US")
for w in words:
    if(d.check(w)==False):
        if(d.suggest(w) is not None):
            try:
                w = d.suggest(w)[0]
            except IndexError:
                w=w